In [50]:
%load_ext autoreload
%autoreload 2

In [98]:
from src.dataset.Loader.main_loader import define_path
from src.dataset.trans.data import TransDataset, extract_pred_sequence
from src.dataset.trans.jaad_trans import JaadTransDataset
from src.dataset.intention.jaad_dataset import build_pedb_dataset_jaad, subsample_and_balance


In [104]:
anns_paths, image_dir = define_path(use_jaad=True, use_pie=False, use_titan=False)
anns_paths_val, image_dir_val = define_path(use_jaad=True, use_pie=False, use_titan=False)

In [105]:
anns_paths

{'JAAD': {'anns': 'DATA/annotations/JAAD/anns/JAAD_DATA.pkl',
  'split': 'DATA/annotations/JAAD/splits/'}}

In [137]:
from src.dataset.loader import PaddedSequenceDataset, IntentionSequenceDataset
from src.transform.preprocess import ImageTransform, Compose, CropBox
import torchvision

In [107]:
bbox_min = 0
max_frames = 5
pred = 10
seed = 99
jitter_ratio = 2.0

crop_preprocess = CropBox(size=224, padding_mode='pad_resize', jitter_ratio=jitter_ratio)
TRAIN_TRANSFORM = Compose([crop_preprocess,
                            ImageTransform(torchvision.transforms.ColorJitter(
                                brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1))
                            ])

train_data = TransDataset(data_paths=anns_paths, image_set="train", verbose=False)
trans_tr = train_data.extract_trans_history(mode='GO', fps=5, max_frames=None,
                                                verbose=True)
non_trans_tr = train_data.extract_non_trans(fps=5, max_frames=None, verbose=True)
sequences_train = extract_pred_sequence(trans=trans_tr, non_trans=non_trans_tr, pred_ahead=pred,
                                            balancing_ratio=1.0, neg_in_trans=True,
                                            bbox_min=bbox_min, max_frames=max_frames, seed=seed, verbose=True)

train_instances = PaddedSequenceDataset(sequences_train, image_dir=image_dir, padded_length=max_frames,
                                            hflip_p = 0.5, preprocess=TRAIN_TRANSFORM)

Extract 1 GO history samples from train dataset,
samples contain 1 unique pedestrians and 61 frames.
Extract Non-transition samples from train dataset  :
Walking: 2 samples,  2 unique pedestrians and 54 frames.
Standing: 0 samples,  0 unique pedestrians and 0 frames.
Extract 59 sequence samples from 1 history.
1/0 ratio:  1 : 4.90
predicting-ahead frames: 10
Perform sample balancing for JAAD:
Orignal samples: P 10 , N 49
Balanced samples:P 10, N 10


In [131]:
for el in train_instances:
    print(el.keys())
    print(el['label'])
    print(el['image'].shape)
    print(el['bbox'])
    break


dict_keys(['image', 'bbox', 'bbox_ped', 'seq_length', 'action', 'id', 'label', 'source', 'TTE', 'TTE_tag', 'behavior', 'attributes'])
tensor(0.)
torch.Size([5, 3, 224, 224])
[[1403.0, 729.0, 1480.0, 855.0], [1413.0, 722.0, 1483.0, 855.0], [1420.0, 717.0, 1482.0, 855.0], [1424.0, 709.0, 1484.0, 859.0], [1425.0, 703.0, 1483.0, 860.0]]


In [44]:
sequences_train['JG_0037_train_f131']

{'source': 'JAAD',
 'video_number': 'video_0194',
 'frame': [107, 113, 119, 125, 131],
 'bbox': [[1079.0, 646.0, 1171.0, 920.0],
  [1090.0, 654.0, 1187.0, 941.0],
  [1101.0, 651.0, 1207.0, 957.0],
  [1111.0, 657.0, 1220.0, 977.0],
  [1120.0, 663.0, 1232.0, 997.0]],
 'action': [0, 0, 0, 0, 0],
 'traffic_light': [0, 0, 0, 0, 0],
 'behavior': [[0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]],
 'attributes': [3, 0, 0, 0, 1, 2],
 'trans_label': 1,
 'TTE': 0.4}

In [91]:
intent_sequences_cropped 

[{'sample_id': '0_114_646b_video_0114_24',
  'frames': [144, 150, 156, 162, 168],
  'bbox': [[960.0, 471.0, 996.0, 562.0],
   [950.0, 472.0, 985.0, 564.0],
   [940.0, 472.0, 975.0, 564.0],
   [929.0, 473.0, 964.0, 566.0],
   [918.0, 474.0, 952.0, 567.0]],
  'action': [1, 1, 1, 1, 1],
  'occlusion': [0, 0, 0, 0, 0],
  'behavior': [[1, 1, 0, 0],
   [1, 1, 0, 0],
   [1, 1, 0, 0],
   [1, 1, 0, 0],
   [1, 1, 0, 0]],
  'traffic_light': [0, 0, 0, 0, 0],
  'label': 0,
  'attributes': [2, 1, 0, 0, 1]},
 {'sample_id': '0_134_797b_video_0134_23',
  'frames': [151, 157, 163, 169, 175],
  'bbox': [[495.0, 643.0, 568.0, 834.0],
   [490.0, 641.0, 552.0, 825.0],
   [477.0, 639.0, 538.0, 829.0],
   [458.0, 640.0, 532.0, 835.0],
   [439.0, 641.0, 525.0, 841.0]],
  'action': [1, 1, 1, 1, 1],
  'occlusion': [1, 0, 0, 0, 0],
  'behavior': [[1, 0, 0, 0],
   [1, 0, 0, 0],
   [1, 0, 0, 0],
   [1, 1, 0, 0],
   [1, 1, 0, 0]],
  'traffic_light': [0, 0, 0, 0, 0],
  'label': 0,
  'attributes': [1, 1, 0, 0, 1]},
 {

In [138]:
max_frames = 10
intent_sequences = build_pedb_dataset_jaad(anns_paths["JAAD"]["anns"], anns_paths["JAAD"]["split"], image_set = "train", fps=5, verbose=True)
intent_sequences_cropped = subsample_and_balance(intent_sequences, max_frames=max_frames)
train_ds = IntentionSequenceDataset(intent_sequences_cropped, image_dir=image_dir, hflip_p = 0.5, preprocess=TRAIN_TRANSFORM)

----------------------------------------------------------------
JAAD:
Total number of crosses: 48
Total number of non-crosses: 112
Filtered samples: 0


In [139]:
for el in train_ds:
    print(el.keys())
    print(el['label'])
    print(el['image'].shape)
    print(el['bbox'])
    print(el['seq_length'])
    break

dict_keys(['image', 'bbox', 'bbox_ped', 'seq_length', 'id', 'label', 'attributes', 'action', 'behavior'])
tensor(1.)
torch.Size([10, 3, 224, 224])
[[883.0, 649.0, 904.0, 711.0], [893.0, 652.0, 912.0, 715.0], [899.0, 654.0, 924.0, 718.0], [905.0, 656.0, 936.0, 720.0], [916.0, 657.0, 951.0, 722.0], [930.0, 657.0, 963.0, 725.0], [945.0, 655.0, 969.0, 726.0], [952.0, 660.0, 989.0, 729.0], [968.0, 661.0, 997.0, 732.0], [982.0, 662.0, 1008.0, 734.0]]
10


In [73]:
for attr in intent_sequences['0_1_3b']:
    if isinstance(intent_sequences['0_1_3b'][attr], list):
        print(attr, len(intent_sequences['0_1_3b'][attr]))

frames 94
bbox 94
action 94
occlusion 94
behavior 94
attributes 1
traffic_light 94
labels 94


In [65]:
len(intent_sequences['0_1_3b']['frames'])

94

In [33]:
image_set = "train"
ds = JaadTransDataset(jaad_anns_path=anns_paths["JAAD"]["anns"],    
                      split_vids_path=anns_paths["JAAD"]["split"],
                      image_set=image_set,
                      verbose=True)

----------------------------------------------------------------
JAAD:
Total number of standing to walking transitions(raw): 82
Total number of walking to standing transitions(raw): 36


In [37]:
a = range(15)[-1:None:-1]
for el in a:
    print(el)

14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [34]:
ds.extract_trans_history(mode="GO")

{'JG_0001_train': {'source': 'JAAD',
  'old_id': '0_1_3b',
  'video_number': 'video_0001',
  'frame': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
  